In [3]:
import pandas as pd
import json

def read_csv(file_path):
    """Read data from the CSV file and return the attributes and rows."""
    try:
        df = pd.read_csv(file_path)
        attributes = df.columns.tolist()
        rows = df.values.tolist()
        return attributes, rows
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None, None

def parse_fd_file(file_path, is_mvd=False):
    """Parse functional and multivalued dependencies from a text file."""
    dependencies = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                parts = line.split('-->>' if is_mvd else '-->')
                if len(parts) != 2:
                    continue
                lhs = [attr.strip() for attr in parts[0].strip('{}').split(',')]
                rhs = [attr.strip() for attr in parts[1].strip('{}').split(',')]
                dependencies.append({"lhs": lhs, "rhs": rhs})
        return dependencies
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []

def create_json_structure(attributes, rows, primary_keys, non_atomic, fds, mvds, target_nf):
    """Create the JSON structure as specified."""
    # Identifying non-atomic attributes (attributes with comma-separated values in rows)

    data_structure = {
        "relations": [
            {
                "table_name": "Coffeeshop_data",
                "attributes": attributes,
                "rows": [],
                "primary_key": primary_keys,
                "candidate_keys": [],  # You can populate this if you have candidate keys
                "prime_attributes": primary_keys,  # Prime attributes are typically the primary keys
                "non_atomic": non_atomic
            }
        ],
        "functional_dependencies": fds,
        "mvds": mvds,
        "target_normal_form": target_nf
    }
    return data_structure

def main():
    # File paths
    csv_file = 'TestingData.csv'
    fd_file = 'fds.txt'
    mvd_file = 'mvds.txt'
    
    # User input for primary keys and target normal form
    primary_keys = input("Enter the primary keys (comma-separated): ").strip().split(',')
    non_atomic = input("Enter the non-atomic keys (comma-separated): ").strip().split(',')
    target_nf = input("Enter the target normal form (e.g., '5NF'): ").strip()

    # Read data from CSV
    attributes, rows = read_csv(csv_file)
    if attributes is None or rows is None:
        return

    # Parse functional dependencies and multivalued dependencies
    fds = parse_fd_file(fd_file)
    mvds = parse_fd_file(mvd_file, is_mvd=True)

    # Create the final JSON structure
    json_structure = create_json_structure(attributes, rows, primary_keys, non_atomic, fds, mvds, target_nf)

    # Output the JSON to a file or print
    with open('output.json', 'w') as outfile:
        json.dump(json_structure, outfile, indent=4)
    print("JSON structure saved to 'output.json'.")

if __name__ == "__main__":
    main()


Enter the primary keys (comma-separated):  OrderID,DrinkID,FoodID
Enter the non-atomic keys (comma-separated):  PromocodeUsed,DrinkIngredient,DrinkAllergen,FoodIngredient,FoodAllergen
Enter the target normal form (e.g., '5NF'):  5NF


JSON structure saved to 'output.json'.
